<a href="https://colab.research.google.com/github/luida-M/Modelo_ML/blob/main/Proyecto_ParteIII%2BMedina_Tenorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
# 1. Configuración Inicial
from google.colab import drive
import os
drive.mount('/content/gdrive')
# Establecer ruta de acceso en drive
print(os.getcwd())
os.chdir("/content/gdrive/My Drive")

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score
import gspread
from google.auth import default
from google.colab import auth
from sklearn.impute import SimpleImputer
from google.colab import drive

# Autenticación con Google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Configurar Git
!git config --global user.email "luidamedina@gmail.com"
!git config --global user.name "Luida Medina"

# Clonar el repositorio
!git clone https://github.com/luida-M/Modelo_ML.git
%cd Modelo_ML

# 2. Carga de Datos
file_name = 'ventas-totales-supermercados-2'  # Nombre del archivo de Google Sheets

try:
    # Abrir la hoja de cálculo por nombre
    sheet = gc.open(file_name).sheet1
    # Obtener todos los datos de la hoja como una lista de listas
    data = sheet.get_all_values()
    # Convertir la lista de listas a un DataFrame de Pandas
    df = pd.DataFrame(data[1:], columns=data[0])
    # Convertir todas las columnas numéricas a tipo de datos numérico
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    # Manejar valores faltantes (opcional, dependiendo de tu estrategia)
    df.fillna(df.mean(), inplace=True)
except gspread.SpreadsheetNotFound:
    print(f"No se encontró el archivo '{file_name}' en Google Sheets. Asegúrate de que el nombre sea correcto.")
except Exception as e:
    print(f"Error al leer el archivo de Google Sheets: {e}")

if 'df' in locals():
    print(df.head())

    # 3. Método de Feature Selection: SelectKBest
    features = ['indice_tiempo','ventas_precios_corrientes', 'ventas_precios_constantes', 'ventas_totales_canal_venta',
                'salon_ventas', 'canales_on_line', 'ventas_totales_medio_pago', 'efectivo', 'tarjetas_debito',
                'tarjetas_credito', 'otros_medios', 'ventas_totales_grupo_articulos', 'subtotal_ventas_alimentos_bebidas',
                'bebidas', 'almacen', 'panaderia', 'lacteos', 'carnes', 'verduleria_fruteria',
                'alimentos_preparados_rotiseria', 'articulos_limpieza_perfumeria', 'indumentaria_calzado_textiles_hogar',
                'electronicos_articulos_hogar', 'otros']

    X = df[features]
    y = df['ventas_precios_corrientes']

    # Manejar valores faltantes con SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    selector = SelectKBest(score_func=f_regression, k=10)
    X_new = selector.fit_transform(X, y)
    selected_features = [features[i] for i in selector.get_support(indices=True)]
    print(f"Selected Features: {', '.join(selected_features)}")

    # Visualización de las características seleccionadas
    plt.figure(figsize=(10, 7))
    plt.barh(selected_features, selector.scores_[selector.get_support()])
    plt.xlabel('Puntuación de Selección')
    plt.ylabel('Características')
    plt.title('Puntuación de las Características Seleccionadas')
    plt.show()

    # 4. Dividir Datos en Entrenamiento y Prueba
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42) # 70% train, 30% test

    # 5. Algoritmos de Regresión
    models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(random_state=42),
        "Ridge Regression": Ridge(),
        "Lasso Regression": Lasso(),
        "ElasticNet Regression": ElasticNet()
    }

    # 6. Entrenamiento y Evaluación
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Visualizar comparaciones reales vs. predicciones
        plt.figure(figsize=(10, 7))
        plt.scatter(y_test, y_pred, color='blue')
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2)
        plt.title(f'Comparación de Valores Reales vs. Predichos: {name}')
        plt.xlabel('Valores Reales')
        plt.ylabel('Valores Predichos')
        plt.show()

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        results[name] = {"MSE": mse, "R2": r2}
        print(f"{name} - Mean Squared Error: {mse}")
        print(f"{name} - R^2 Score: {r2}")

    # Visualización de Resultados
    mse_values = [results[name]["MSE"] for name in results]
    r2_values = [results[name]["R2"] for name in results]

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.barh(list(results.keys()), mse_values, color='skyblue')
    plt.xlabel('Mean Squared Error (MSE)')
    plt.title('Comparación de MSE entre Algoritmos')

    plt.subplot(1, 2, 2)
    plt.barh(list(results.keys()), r2_values, color='lightgreen')
    plt.xlabel('R^2 Score')
    plt.title('Comparación de R^2 entre Algoritmos')

    plt.tight_layout()
    plt.show()

    # 7. Generación de Conclusiones
    mejores_resultados = sorted(results.items(), key=lambda x: x[1]["MSE"])[0]
    mejor_algoritmo = mejores_resultados[0]
    mejor_mse = mejores_resultados[1]["MSE"]
    mejor_r2 = mejores_resultados[1]["R2"]

    conclusiones = f"""
    1. Las características seleccionadas para el modelo son: {', '.join(selected_features)}.
    2. El mejor algoritmo de regresión fue {mejor_algoritmo}, con un error cuadrático medio (MSE) de {mejor_mse} y un R^2 Score de {mejor_r2}.
    3. Las características seleccionadas han mostrado ser relevantes para predecir 'ventas_precios_corrientes'.
    4. Se recomienda utilizar el algoritmo {mejor_algoritmo} para predicciones futuras, ya que presentó el mejor desempeño en términos de MSE y R^2 Score.
    """
    print(conclusiones)
else:
    print("No se puede proceder con el análisis ya que el archivo no se cargó correctamente.")

    # Subir cambios a GitHub
    !git add . !git commit -m "Análisis de datos y comparación de varios algoritmos de regresión"
    !git push origin main



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/gdrive
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-54a707d7230b>", line 8, in <cell line: 8>
    print(os.getcwd())
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/py